In [120]:
import hashlib
import numpy as np
import statistics
import math
import collections
from zlib import crc32
import random
import operator
import scipy.stats as stats
from Crypto.Util import number

In [121]:
#helper function
def expo_of_two(num):
    result = num & (num-1)
    return result == 0


In [122]:
class CSSS_CountSketch():
    def __init__(self, d, t):
        # original count sketch t = O(1/epsilon) and d = O(log(1/delta)) 
        # think delta as U^{-c} d = O(logU)
        # d represent number of rows and t represent number of columns
        
        self.total_input = 0
        self.columns = t * 6
        self.rows = d
        self.prime = number.getPrime(32)
        
        self.table_positive = np.zeros( (self.rows, self.columns) )
        self.table_negative = np.zeros( (self.rows, self.columns) )
        
        # generate 4-wise independent hash functions
        self.a = []
        self.b = []
        self.c = []
        self.d = []
        for i in range(d):
            aj, bj = np.random.randint(self.prime - 1, size=2) #randomly select one number from 0 to self.prime -1
            cj, dj = np.random.randint(self.prime - 1, size=2)
            self.a.append(aj+1)
            self.b.append(bj+1)
            self.c.append(cj+1)
            self.d.append(dj+1)
    def routine_a(self):
        for i in range(self.rows):
            for j in range(self.columns):
                self.table_positive[i][j] = np.random.binomial(self.table_positive[i][j], 1/2)
                self.table_negative[i][j] = np.random.binomial(self.table_negative[i][j], 1/2)

    def update(self, item, weight=1):
        item = int(item)
        self.total_input += weight

        for j in range(self.rows):
            hj = ( self.a[j] * item + self.b[j] % self.prime) % self.columns
            gj = 2 * ((self.c[j] * item + self.d[j] % self.prime) % 2) - 1
            assert( gj == -1 or gj == 1)
            result = weight * gj
            if result > 0:
                self.table_positive[j][hj] += result
            else:
                self.table_negative[j][hj] += -1*result

    def query(self, item):
        item = int(item)
        ans = []
        for j in range(self.rows):
            hj = int( (self.a[j] * item + self.b[j] % self.prime) % self.columns )
            gj = 2 * ((self.c[j] * item + self.d[j] % self.prime) % 2) - 1
            assert( gj == -1 or gj == 1)
            ans.append ( gj * (self.table_positive[j][hj] - self.table_negative[j][hj] ))

        return statistics.median(ans)

    def inputsize(self):
        return self.total_input

    def countertable(self):
        return self.table

In [123]:
class CSSS_sketch():
    def __init__(self, epsilon=0.01, universe=2**16, k=10, alpha=2):
        assert k>=1
        assert 0<epsilon<1
        self.T = math.ceil(4/(epsilon**2) + math.log2(universe))
        self.S = math.ceil( alpha*alpha/(epsilon*epsilon) * self.T*self.T* math.log2(1/epsilon) )
        self.d = math.ceil(math.log2(universe))
        self.multiple = math.ceil(math.log2(self.S)) # used to perform binomial sampling
        
        self.k = k
        self.count_sketch = CSSS_CountSketch( self.d, self.k)
        self.r = 1
        self.p = 0
        self.time = 0
    def update(self, item, weight=1):
        self.time += 1
        factor = (self.time -1) /self.multiple
        if (self.time -1)%self.multiple == 0 and expo_of_two(int(factor)) :
            self.count_sketch.routine_a()
        rand = random.random()
        if rand < 2**(-1*self.p):
            #sampled
            self.p+=1
            self.count_sketch.update(item, weight)
    def query(self, item):
        ans = self.count_sketch.query(item)
        return ans* (2**(self.p))

In [128]:
    universe = 2**16
    size = 1000
    R = size # assume we can capture R without any error
    alpha = 2
    rate = 0.5
    epsilon = 0.1
    k = math.ceil( 32/epsilon)
    epsilon_prime = epsilon/32.0
    CSSS = CSSS_sketch(epsilon_prime, universe, k, alpha)
    
    a = 2.0 #skewness
    range_x = np.arange(1,universe)
    weights = range_x ** (-a)
    weights /= weights.sum()
    bounded_zipf = stats.rv_discrete(name='bounded_zipf', values=(range_x, weights))
    insertions = bounded_zipf.rvs(size=size)   
    deletions = random.sample(list(insertions), int(rate*size))
    true_result = {}
    for i in insertions:
        CSSS.update(str(i), 1)
        true_result[str(i)] = true_result.get(str(i), 0) + 1
    for i in deletions:
        CSSS.update(str(i), -1)
        true_result[str(i)] = true_result.get(str(i), 0) - 1

    max_error = 0
    avg_error = 0
    top_k = {}
    for item in true_result.keys():
        estimate = CSSS.query(str(item))
        error = abs(true_result.get(str(item),0) - estimate)
        if abs(estimate) >= 3*epsilon*R/4.0:
            top_k[ str(item)] = CSSS.query(str(item))
        max_error = max(max_error, error)
        avg_error += error

    avg_error = 1.0*avg_error / len(true_result.keys())
    print("Estimate Heavy Hitters")
    print("With high probablity, it contains all item with occurance more than "+str(epsilon*size)+" , and no item occured less than "+str(epsilon/2*size))
    sorted_top_k = sorted(top_k.items(), key=operator.itemgetter(1),reverse=True)
    print(sorted_top_k)

    print("true top "+str( int(1/epsilon)))
    true_top_k = sorted(true_result.items(), key=operator.itemgetter(1),reverse=True)
    print(true_top_k[:int(1/epsilon)])

    print("------------------------------------------------------------------------------")
    print("maximum error is: ")
    print(max_error)
    print("average error is: ")
    print(avg_error)


Estimate Heavy Hitters
With high probablity, it contains all item with occurance more than 100.0 , and no item occured less than 50.0
[('1', 1024.0)]
true top 10
[('1', 309), ('2', 90), ('3', 34), ('5', 10), ('7', 9), ('4', 8), ('6', 8), ('10', 5), ('16', 5), ('9', 3)]
------------------------------------------------------------------------------
maximum error is: 
715.0
average error is: 
23.23076923076923
